In [1]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display as Markdown
from langchain_community.document_loaders import TextLoader, PyPDFLoader


load_dotenv()
api = os.getenv('HUGGING_FACE_API_KEY')

In [8]:
loader = PyPDFLoader('./pdf/what-is-generative-ai.pdf')
pdf_file = loader.load()
pdf_file

[Document(metadata={'source': './pdf/what-is-generative-ai.pdf', 'page': 0}, page_content='January 2023\nMcKinsey Explainers\nWhat is generative AI?\nGenerative artificial intelligence (AI) describes algorithms (such \nas ChatGPT) that can be used to create new content, including \naudio, code, images, text, simulations, and videos. Recent new \nbreakthroughs in the field have the potential to drastically change \nthe way we approach content creation.'),
 Document(metadata={'source': './pdf/what-is-generative-ai.pdf', 'page': 1}, page_content='Generative AI systems fall under the broad \ncategory of machine learning, and here’s how one \nsuch system—ChatGPT—describes what it can do:\nReady to take your creativity to the next level? \nLook no further than generative AI! This nifty form \nof machine learning allows computers to generate \nall sorts of new and exciting content, from music \nand art to entire virtual worlds. And it’s not just for \nfun—generative AI has plenty of practical

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_split = RecursiveCharacterTextSplitter(chunk_size=500,
                                            chunk_overlap=100)
docs = text_split.split_documents(pdf_file)
docs = docs[:3]

In [4]:
import requests

API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
headers = {"Authorization": f"Bearer {api}"}


def query(text):
  payload = {"inputs": text}
  responnse = requests.post(API_URL, headers=headers, json=payload)
  return responnse.json()

In [9]:
def get_embedding(text):
    # Memastikan bahwa input adalah string
    if not isinstance(text, str):
        print(f"Invalid input: Expected string, but got {type(text)}")
        return None

    # Batas panjang teks yang direkomendasikan (opsional, tergantung kebutuhan)
    max_length = 512  # Misalnya, panjang maksimum bisa disesuaikan
    if len(text) > max_length:
        text = text[:max_length]  # Potong teks jika terlalu panjang

    # Mengirimkan teks langsung tanpa 'source_sentence' atau 'sentences'
    payload = {"inputs": text}
    output = query(payload)

    # Cek apakah output mengandung error
    if 'error' in output:
        print(f"Error from API: {output['error']}")
        return None

    # Cek apakah output adalah list angka (embedding)
    if isinstance(output, list) and all(isinstance(i, float) for i in output):
        return output  # Mengembalikan embedding mentah

    print('Unexpected format:', output)
    return None


# Proses embedding untuk setiap dokumen
all_embeddings = []
for document in docs:
    text = document.page_content.strip()  # Menghapus karakter kosong di awal/akhir
    embedding = get_embedding(text)

    if embedding is not None:
        all_embeddings.append(embedding)

Error from API: ['Field required: received `source_sentence` in `parameters`', 'Field required: received `sentences` in `parameters`']
Error from API: ['Field required: received `source_sentence` in `parameters`', 'Field required: received `sentences` in `parameters`']
Error from API: ['Field required: received `source_sentence` in `parameters`', 'Field required: received `sentences` in `parameters`']
